In [1]:
import pandas as pd
from collections import defaultdict
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import stopwords
import re

In [2]:
def conv_column_arr(data_frame, field, field_arr):
    list_f_arr = []
    dict_hasil = {}
    f_arr = data_frame[field_arr]
    for i in f_arr:
        deleted_simbol = i.replace(" ","").replace("'", "").replace("[","").replace("]","")
        temp = deleted_simbol.split(",")
        
#         dict_hasil[term[i]] = num_doc
        list_f_arr.append(temp)
    dict_hasil = {field: data_frame[field], field_arr:list_f_arr }
    df = pd.DataFrame(dict_hasil)
    
    return df

# Baca File CSV yang Di Perlukan

In [3]:
df_doc_num_token = pd.read_csv("doc_num_token.csv")
df_doc_num_token = conv_column_arr(df_doc_num_token,"doc_num","token")
df_doc_num_token.head()

,doc_num,token
0,1,"[0.39, 1, 750, 1, 780, 1, 850, 1, 870, 1, 875,..."
1,2,"[15:02:20.00, 26-feb-1987, 55, activ, also, am..."
2,3,"[13.5, 15:03:27.51, 26-feb-1987, 31, 7.5, appl..."
3,4,"[-1, -20, 1, 11, 12, 15, 15:07:13.72, 2, 2.70,..."
4,5,"[-1986, 0.99, 1, 1.24, 1.35, 1.56, 1.65, 1.92,..."


In [4]:
df_token_doc_num = pd.read_csv("hasil_preprocessing.csv")
df_token_doc_num = conv_column_arr(df_token_doc_num, "term", "doc_num")
df_token_doc_num.head()

,term,doc_num
0,-1,"[0004, 0008, 0016, 0026, 0059, 0059, 0059, 005..."
1,-10,[0305]
2,-100,[0427]
3,-11,[0116]
4,-110,[0427]


In [5]:
# menghitung jml token
jml_token = 0
for i in range(len(df_doc_num_token)):
    jml_token += len(df_doc_num_token["token"].iloc[i])
jml_token

49088

# Fungsi-fungsi untuk preprocessing query

In [6]:
def make_token(txt):
    list_hasil = txt.split(" ")            
    return list_hasil

In [7]:
def removeNull(listOfWords): #['','','eat','food','','']
    listOfWords = list(filter(None, listOfWords))
    
    return listOfWords

In [8]:
def caseFolding(listOfWords):
    for i in range(len(listOfWords)):
        listOfWords[i] = listOfWords[i].casefold()
        
    return listOfWords

In [9]:
stop_words = stopwords.words('english')

In [10]:
def removeStopWord(listOfWords):
    for i in listOfWords:
        if i in stop_words:
            listOfWords.remove(i)
        
    return listOfWords

In [11]:
def stemming(listOfWords):
    stemmer = SnowballStemmer("english")
    stemmed = [stemmer.stem(word) for word in listOfWords]
    
    return stemmed

### Fungsi utama main preprocessing query

In [12]:
def pre_query(txt):
    listOfWords = make_token(txt)
    listOfWords = removeNull(listOfWords)
    listOfWords = caseFolding(listOfWords)
    listOfWords = removeStopWord(listOfWords)
    listOfWords = stemming(listOfWords)
    
    return listOfWords

# Fungsi utama menghitung Score dokumen

In [13]:
def hitung_score(list_word,a):
    list_rank = []
    list_doc_num2 = []
    df_hasil = []
    for i in range(len(df_doc_num_token)):
        rank = 1
        doc = df_doc_num_token.iloc[i]
        for j in list_word:
            count = doc["token"].count(j)
            panjang_doc = len(doc["token"])
            p_doc =  (count/panjang_doc) * a
#           ----------------------------------
            temp = df_token_doc_num[df_token_doc_num["term"] == j]
#             print(i, temp)
            count_m = len(temp.iloc[0][1])
            p_m = (count_m/jml_token) * (1-a)
            p = p_doc + p_m
            rank *= p
        list_rank.append(rank)
        list_doc_num2.append(doc["doc_num"])
    dict_hasil = {"doc_num": list_doc_num2, "rank": list_rank}
    df_hasil = pd.DataFrame(dict_hasil)
    df_hasil = df_hasil.sort_values(by=["rank"], ignore_index=True, ascending=False)
    return df_hasil
        

# Coba query

In [14]:
q1 = "1991"
q2 = "Dicounts"
q3 = "accessed 9-Mar-87"
q4 = "Analyst appeared"
q5 = "Analyst analyst appeared"
q6 = "Chairman closed the last"

In [15]:
q = pre_query(q1)
q

['1991']

In [16]:
df_hasil = hitung_score(q,0.1)

In [17]:
jawaban1 = [31, 61, 70, 105, 190, 301, 422, 466]
jawaban2 = [77, 80, 97, 160, 195, 208, 215, 256, 488]
jawaban3 = [32, 55, 110, 116, 134, 141, 142, 143, 144, 145, 146, 147, 148,
            149, 150, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 261, 
            262, 263, 264, 265, 266, 267, 268, 269, 270, 321, 322, 323, 324, 
            325, 326, 327, 328, 329, 330, 348, 375, 381, 382, 383, 384, 385, 
            386, 387, 388, 389, 390, 441, 442, 443, 444, 445, 446, 447, 448,
            449, 450]
jawaban4 = [4, 8, 16, 18, 28, 45, 78, 95, 98, 111, 123, 133, 150, 180, 195, 
            235, 243, 262, 265, 292, 305, 320, 376, 399, 418, 459, 469, 493, 
            495, 496, 497]
jawaban5 = [4, 8, 16, 18, 28, 45, 78, 95, 98, 111, 123, 133, 150, 180, 195, 
            235, 243, 262, 265, 292, 305, 320, 376, 399, 418, 459, 469, 493, 
            495, 496, 497]
jawaban6 = [1, 12, 18, 26, 28, 40, 43, 47, 52, 60, 61, 69, 78, 95, 98, 102, 
           107, 111, 114, 116, 133, 134, 135, 136, 138, 145, 148, 150, 153, 
           154, 155, 156, 161, 169, 174, 178, 180, 188, 194, 195, 197, 201, 
           203, 204, 207, 208, 214, 216, 224, 227, 230, 231, 232, 233, 234, 
           235, 242, 246, 254, 255, 256, 259, 261, 262, 263, 265, 267, 268, 
           269, 276, 279, 284, 288, 292, 303, 306, 314, 316, 318, 322, 324, 
           328, 330, 338, 339, 340, 343, 350, 355, 367, 371, 372, 375, 376, 
           378, 382, 383, 386, 392, 399, 404, 405, 409, 419, 420, 425, 430, 
           438, 439, 441, 444, 445, 446, 447, 448, 451, 454, 455, 457, 458, 
           463, 484, 488, 491, 492, 494, 495, 496, 498]

In [18]:
teratas = df_hasil.head(len(jawaban1)).sort_values(by=["doc_num"], ignore_index=True, ascending=True)


In [19]:
hasil_query = teratas["doc_num"]
salah = []

for i in hasil_query:
    if int(i) not in jawaban1:
        salah.append(i)
        
len(salah)

0

In [20]:
teratas

,doc_num,rank
0,31,0.005721
1,61,0.000714
2,70,0.001218
3,105,0.001516
4,190,0.002438
5,301,0.001728
6,422,0.001062
7,466,0.000959


In [21]:
from flask import Flask, request, render_template, url_for, redirect, jsonify

In [22]:
def search_query(search, res):
    res2 = int(res)
    list_hasil = []
    q = pre_query(search)
    df_hasil = hitung_score(q,0.1)
    for i in range(0, res2):
        list_hasil.append(df_hasil.iloc[i]["doc_num"])
    return list_hasil

In [23]:
def readText(file):
    words = []
    f = open(file, 'r') #open file
    text = f.read()    
    f.close()

    return text

In [27]:
def get_atribut(doc_id):
    int_doc_id = int(doc_id)
    str_doc_id = str(int_doc_id)
    n = 4 - len(str_doc_id)
    doc = ("0"*n) + str_doc_id
    file_path = "DataRouter\\Doc" + doc + ".txt"
    
    txt = readText(file_path)
    
    titlePattern = "<TITLE>(.+)</TITLE>"
    title = re.findall(titlePattern, txt)[0]
    
    datePattern = "<DATE>(.+)</DATE>"
    date = re.findall(datePattern, txt)[0]
    
    return title, date

In [ ]:
ui = Flask(__name__)
x = []
@ui.route('/')
def index():
    return render_template('base.html')


@ui.route('/search', methods=['POST', 'GET'])
def search():
    if request.method == 'POST':
        list_title = []
        list_date = []
        search = request.form["search"]
        method = request.form.get('met')
        result = request.form.get('res')
        hasil = search_query(search, result)
        for i in hasil:
            title, date = get_atribut(i)
            list_title.append(title)
            list_date.append(date)
        return render_template('respond.html', title = list_title, date = list_date)
    else:
        return render_template('index.html')
    
@ui.route("/<rsp>/<met>/<res>", methods=['POST', 'GET'])
def respon(rsp, met, res):
    
    return render_template('respond.html', respon = rsp)

if __name__ == "__main__":
    ui.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [21/Apr/2020 15:58:15] "GET /search HTTP/1.1" 200 -
127.0.0.1 - - [21/Apr/2020 15:58:27] "POST /search HTTP/1.1" 200 -
